In [1]:
import os
os.chdir('../')

In [2]:
%pwd

'/workspaces/Xray'

In [3]:
from src.utils.utils import read_yaml,create_dir
from src.constant.ymal_path import *
from src.logging.logger import logging
from src.exception.exception import CustomException
import sys

2024-05-04 18:04:04.537047: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-04 18:04:05.468431: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainConfig:
    dir: Path
    model:Path
    model_path:Path
    train_data:Path
    test_data:Path
    eposhs:int
    batch_size:int 
    image_size:list
    AUGMENTATION: bool

In [5]:
class ConfigManger:
    def __init__(self,
                 config_file_path=config_file_path,
                 params_file_path=params_file_path) -> None:
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)

        create_dir([self.config.artifacts_root])

    def get_model_train_config(self):

        config=self.config.model_train
        train_data=os.path.join('artifacts/data_ingestion/data/train')
        test_data=os.path.join('artifacts/data_ingestion/data/test')

        model_train_config=ModelTrainConfig(
            dir=config.dir,
            model=config.model,
            model_path=config.model_path,
            train_data=train_data,
            test_data=test_data,
            eposhs=self.params.EPOCHS,
            batch_size=self.params.BATCH_SIZE,
            image_size=self.params.IMAGE_SIZE,
            AUGMENTATION=self.params.AUGMENTATION
        )
        return model_train_config

In [6]:
from tensorflow.keras.models import load_model

In [7]:
import tensorflow as tf

In [8]:
from src.utils.utils import save_model

In [11]:
class ModelTrain:
    def __init__(self,config:ModelTrainConfig) -> None:
        self.config=config

    def get_model(self):
        self.model=load_model(self.config.model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def validation_data(self):
        try:
            datagen_kwargs=dict(
                rescale=1./255
            )
       
            dataflow_kwargs = dict(
            target_size=self.config.image_size[:-1],
            batch_size=self.config.batch_size,
            interpolation="bilinear"
        )       

            if self.config.AUGMENTATION:
                datagen_kwargs.update(
                    rotation_range=40,
                    horizontal_flip=True,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    shear_range=0.2,
                    zoom_range=0.2
                )

            train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
            self.train_genarator=train_datagen.flow_from_directory(
                directory=self.config.train_data,
                subset='training',
                shuffle=True,
                **dataflow_kwargs
            )

            test_datagen=tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
            self.validation_genarator=test_datagen.flow_from_directory(
                directory=self.config.test_data,
                shuffle=False,
                **dataflow_kwargs
            )
        except Exception as e:
            logging.info('errorr ',str(e))
            raise CustomException(sys,e)
        
    def train(self):
        self.validation_per_steps=self.validation_genarator.samples//self.validation_genarator.batch_size

        opt=tf.keras.optimizers.SGD(learning_rate=0.01)
        loss=tf.keras.losses.categorical_crossentropy

        self.model.compile(optimizer=opt,loss=loss,metrics=['accuracy'])

        self.model.fit(
            self.train_genarator,
            epochs=self.config.eposhs,
            validation_data=self.validation_genarator
        )

        self.save_model(
                        path=self.config.model_path,model=self.model)
    

In [12]:
try:
    config=ConfigManger()
    model_train_config=config.get_model_train_config()
    model_train=ModelTrain(config=model_train_config)
    model_train.get_model()
    model_train.validation_data()
    model_train.train()

except Exception as e:
            logging.info('errorr ',str(e))
            raise CustomException(sys,e)

Found 210 images belonging to 2 classes.
Found 60 images belonging to 2 classes.


Epoch 1/3


/home/codespace/.local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-05-04 18:04:26.309311: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 403734528 exceeds 10% of free system memory.
2024-05-04 18:04:28.071952: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 201867264 exceeds 10% of free system memory.
2024-05-04 18:04:28.072878: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 403734528 exceeds 10% of free system memory.


 1/14 ━━━━━━━━━━━━━━━━━━━━ 1:14 6s/step - accuracy: 0.5000 - loss: 0.6860

2024-05-04 18:04:28.744466: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 403734528 exceeds 10% of free system memory.
2024-05-04 18:04:30.533768: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 201867264 exceeds 10% of free system memory.


14/14 ━━━━━━━━━━━━━━━━━━━━ 39s 3s/step - accuracy: 0.4765 - loss: 0.6922 - val_accuracy: 0.6000 - val_loss: 0.6864
Epoch 2/3
14/14 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.5955 - loss: 0.6908 - val_accuracy: 0.6167 - val_loss: 0.6883
Epoch 3/3
14/14 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - accuracy: 0.5736 - loss: 0.6884 - val_accuracy: 0.7167 - val_loss: 0.6836
